In [1]:
source = {1, 2, 3, 4, 5, 6, 7, 8}
target = {1, 2, 3, 4, 5, 6, 7, 8}
dist = {(1, 1): 0, (1, 2): 11, (1, 3): 5, (1, 4): 12, (1, 5): 7, (1, 6): 13, (1, 7): 9, (1, 8): 11,
          (2, 2): 0, (2, 3): 13, (2, 4): 2, (2, 5): 17, (2, 6): 4, (2, 7): 15, (2, 8): 20,
          (3, 3): 0, (3, 4): 14, (3, 5): 1, (3, 6): 15, (3, 7): 10, (3, 8): 12,
          (4, 4): 0, (4, 5): 18, (4, 6): 5, (4, 7): 16, (4, 8): 21,
          (5, 5): 0, (5, 6): 20, (5, 7): 15, (5, 8): 17,
          (6, 6): 0, (6, 7): 19, (6, 8): 22,
          (7, 7): 0, (7, 8): 30,
          (8, 8): 0}
# cluster = {p: set() for p in source}
clusters = [set() for p in source]
eps = 10
MinPts = 3

In [2]:
def DBSCAN(source, target, cluster):
    for p in source:
        neighbors = set()
        for q in target:
            if dist[min(p, q), max(p, q)] <= eps:
                neighbors.add(q)
        if len(neighbors) >= MinPts:
            target = source.copy()
            target.remove(p)
            cluster[p].update(neighbors)
            cluster[p].update(DBSCAN(source=neighbors, target=target, cluster=cluster)[p])
    return cluster

In [3]:
def DBSCAN(source, target, clusters):
    cluster_count = 0
    for p in source:
        neighbors = set()
        # cluster = set()
        for q in target:
            if dist[min(p, q), max(p, q)] <= eps:
                neighbors.add(q)
        if len(neighbors) >= MinPts:
            new_target = target.difference(neighbors)
            clusters[cluster_count].update(neighbors)
            neighbors.remove(p)
            clusters[cluster_count].update(DBSCAN(source=neighbors, target=new_target, clusters=clusters)[cluster_count])
            cluster_count += 1
    return clusters

In [2]:
class DBSCAN():
    def __init__(self, eps=10, MinPts=3):
        self.eps = eps
        self.MinPts = MinPts
    
    def get_neighbors(self, point, data, dist):
        neighbors = []
        for candidate in data:
            if dist[min(point, candidate), max(point, candidate)] <= self.eps:
                neighbors.append(candidate)
        return neighbors
    
    def fit(self, data, dist):
        print('Initializing fit.')
        cluster_id = 0
        print('Current cluster id: {}'.format(cluster_id))
        labels = ['undefined'] * len(data)
        for p in data:
            print('Examining point {}'.format(p))
            if labels[p - 1] != 'undefined':
                print('Point {} visited.'.format(p))
                continue
            neighbors = self.get_neighbors(p, data, dist)
            print('Neighbors of point {}: {}'.format(p, neighbors))
            if len(neighbors) < self.MinPts:
                labels[p - 1] = 'noise'
                continue
            cluster_id += 1
            labels[p - 1] = cluster_id
            print('Label for point {}: {}'.format(p, cluster_id))
            index = 0
            while index < len(neighbors):
                neighbor = neighbors[index]
                print('Current neighbor for {}: {}'.format(p, neighbor))
                print('Current labels list: {}'.format(labels))
                if labels[neighbor - 1] == 'noise':
                    labels[neighbor - 1] = cluster_id
                if labels[neighbor - 1] != 'undefined':
                    index += 1
                    continue
                print('Neighbor {} not visited.'.format(neighbor))
                labels[neighbor - 1] = cluster_id
                next_neighbors = self.get_neighbors(neighbor, data, dist)
                if len(next_neighbors) >= self.MinPts:
                    neighbors += next_neighbors
                    print('Neighbors for {}: {}'.format(neighbor, next_neighbors))
                    print('Extended neighbors: {}'.format(neighbors))
                index += 1
        return labels

In [3]:
dbscan = DBSCAN(eps=eps, MinPts=MinPts)
# dbscan.get_neighbors(8, data=source, dist=dist)
dbscan.fit(data=source, dist=dist)

Initializing fit.
Current cluster id: 0
Examining point 1
Neighbors of point 1: [1, 3, 5, 7]
Label for point 1: 1
Current neighbor for 1: 1
Current labels list: [1, 'undefined', 'undefined', 'undefined', 'undefined', 'undefined', 'undefined', 'undefined']
Current neighbor for 1: 3
Current labels list: [1, 'undefined', 'undefined', 'undefined', 'undefined', 'undefined', 'undefined', 'undefined']
Neighbor 3 not visited.
Neighbors for 3: [1, 3, 5, 7]
Extended neighbors: [1, 3, 5, 7, 1, 3, 5, 7]
Current neighbor for 1: 5
Current labels list: [1, 'undefined', 1, 'undefined', 'undefined', 'undefined', 'undefined', 'undefined']
Neighbor 5 not visited.
Neighbors for 5: [1, 3, 5]
Extended neighbors: [1, 3, 5, 7, 1, 3, 5, 7, 1, 3, 5]
Current neighbor for 1: 7
Current labels list: [1, 'undefined', 1, 'undefined', 1, 'undefined', 'undefined', 'undefined']
Neighbor 7 not visited.
Neighbors for 7: [1, 3, 7]
Extended neighbors: [1, 3, 5, 7, 1, 3, 5, 7, 1, 3, 5, 1, 3, 7]
Current neighbor for 1: 1
Curr

[1, 2, 1, 2, 1, 2, 1, 'noise']